In [136]:
import pandas as pd
import numpy
import pylab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import LinearRegression

ImportError: cannot import name LinearRegression

In [137]:
trainBodies = pd.read_csv('fncData/train_bodies.csv')
trainStances = pd.read_csv('fncData/train_stances.csv')
trainStancesRandom = pd.read_csv('fncData/train_stances.random.csv')

print "Train Bodies (20/%s):" % (len(trainBodies))
#print trainBodies.head(20)

# to get articleBody:
# trainBodies.get_value(0, 'articleBody')

Train Bodies (20/1683):


In [138]:
limit = len(trainStances)
limit = 1000
strip = True
lowercase = True

In [139]:
print "Train Stances (20/%s):" % (len(trainStances))
# trainStances.head(20)

trainStances["unrelated"] = trainStances["Stance"].apply(lambda x: 1 if x == "unrelated" else 0)
trainStances["agree"] = trainStances["Stance"].apply(lambda x: 1 if x == "agree" else 0)
trainStances["disagree"] = trainStances["Stance"].apply(lambda x: 1 if x == "disagree" else 0)
trainStances["discuss"] = trainStances["Stance"].apply(lambda x: 1 if x == "discuss" else 0)
trainStances["Body"] = [""] * len(trainStances)
for index, row in trainStances[0:limit].iterrows():
    trainStances.loc[index, "Body"] = trainBodies[trainBodies['Body ID'] == row["Body ID"]]['articleBody'].item()


import re
def preprocess(text, lowercase, strip):
    text = text.lower() if lowercase else text
    text = " ".join(re.findall("[a-zA-Z]+", text)) if strip else text
    return text

if strip or lowercase:
    trainStances["Body"]= trainStances["Body"].apply(lambda x: preprocess(x,lowercase,strip))
    trainStances["Headline"]= trainStances["Headline"].apply(lambda x: preprocess(x,lowercase,strip))
    

trainStances = trainStances.drop("Body ID", 1)
trainStances = trainStances.drop("Stance", 1)
    
trainStances.head()

Train Stances (20/49972):


,Headline,unrelated,agree,disagree,discuss,Body
0,police find mass graves with at least bodies n...,1,0,0,0,danny boyle is directing the untitled film set...
1,hundreds of palestinians flee floods in gaza a...,0,1,0,0,hundreds of palestinians were evacuated from t...
2,christian bale passes on role of steve jobs ac...,1,0,0,0,year old moscow resident was hospitalized with...
3,hbo and apple in talks for month apple tv stre...,1,0,0,0,reuters a canadian soldier was shot at the can...
4,spider burrowed through tourist s stomach and ...,0,0,1,0,fear not arachnophobes the story of bunbury s ...


In [140]:
print "Train Stances Random (20/%s)" % (len(trainStancesRandom))
# trainStancesRandom.head(20)

Train Stances Random (20/49972)


In [141]:
# build feature vocabulary
ngram = 1
vectorizer = CountVectorizer(ngram_range=(1, ngram))
trainStances["Number Of Common Words"] = [-1 for x in range(len(trainStances))]

for index, row in trainStances[0:limit].iterrows():
    headline = row['Headline']
    body = row['Body']
    
#     print index
#     print "HEADLINE: \n%s\n" % headline
#     print "BODY: \n%s\n" % body

    headline_words = headline.split()
    body_words = body.split()
    
    num_words_in_common = len(set(headline_words).intersection(body_words))
    
    trainStances.loc[index, "Number Of Common Words"] = num_words_in_common 

trainStances.head()

# I'd like to graph the num_words_in_common integer to the stance classification, to see what kind of correlation we're working with here.

,Headline,unrelated,agree,disagree,discuss,Body,Number Of Common Words
0,police find mass graves with at least bodies n...,1,0,0,0,danny boyle is directing the untitled film set...,2
1,hundreds of palestinians flee floods in gaza a...,0,1,0,0,hundreds of palestinians were evacuated from t...,10
2,christian bale passes on role of steve jobs ac...,1,0,0,0,year old moscow resident was hospitalized with...,4
3,hbo and apple in talks for month apple tv stre...,1,0,0,0,reuters a canadian soldier was shot at the can...,2
4,spider burrowed through tourist s stomach and ...,0,0,1,0,fear not arachnophobes the story of bunbury s ...,9


In [142]:
trainStances[0:limit].corr()

,unrelated,agree,disagree,discuss,Number Of Common Words
unrelated,1.000000,-0.495683,-0.207277,-0.738470,-0.666706
agree,-0.495683,1.000000,-0.041357,-0.147343,0.342553
disagree,-0.207277,-0.041357,1.000000,-0.061614,0.115451
discuss,-0.738470,-0.147343,-0.061614,1.000000,0.490998
Number Of Common Words,-0.666706,0.342553,0.115451,0.490998,1.000000


In [147]:
categoryNumberOfCommonWordsMeans = {
    "unrelated":0, "discuss":0, "agree":0, "disagree":0
}
for category in categoryNumberOfCommonWordsMeans: #for each category calculate the mean number of words in common between header and body 
    categoryNumberOfCommonWordsMeans[category] = trainStances[0:limit][trainStances[category] == 1].mean()["Number Of Common Words"]

print(categoryNumberOfCommonWordsMeans)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


{'unrelated': 2.6227208976157081, 'disagree': 7.0, 'discuss': 7.572222222222222, 'agree': 7.7111111111111112}


In [148]:
from sklearn.feature_extraction.text import CountVectorizer
bodies = [x for x in trainStances[0:limit]["Body"]]
headlines = [x for x in trainStances[0:limit]["Headline"]]

# could make count vectors for each type
bodyBOWVectors = CountVectorizer(analyzer="word").fit_transform(bodies)
headlineBOWVectors = CountVectorizer(analyzer="word").fit_transform(headlines)

# maybe its also OK to have one vectorizer where the presence of the word is scaled up if that word is present in the headline

212
